# Read Summarization or Read Counts

<big><b>File under construction... </b></big> (05/28/2021) <br>

---

## <b>Preparing session for IFB core cluster</b>

<em>loaded JupyterLab</em> : Version 2.2.9

In [3]:
echo "=== Cell launched on $(date) ==="

echo "=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ==="
jobid=$(squeue -hu $USER | awk '/jupyter/ {print $1}')
sacct --format=JobID,AllocCPUS,NODELIST -j ${jobid}

echo "=== Working's root folder is ==="
gohome="/shared/projects/gonseq/Building/" # to adjust with your project's folder
echo "${gohome}"
echo ""

echo "=== current folder tree ==="
tree -d -L 2 "${gohome}"
echo "=== current working directory ==="
echo "${PWD}"

=== Cell launched on Mon May 31 14:41:46 CEST 2021 ===
=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ===
       JobID  AllocCPUS        NodeList 
------------ ---------- --------------- 
16989888             10     cpu-node-47 
16989888.ba+         10     cpu-node-47 
16989888.0           10     cpu-node-47 
=== Working's root folder is ===
/shared/projects/gonseq/Building/

=== current folder tree ===
/shared/projects/gonseq/Building/
├── Data
│   ├── fastq
│   ├── info
│   └── sra
├── Pipeline
├── Reference
│   ├── extracted
│   └── indexes_upto49bases
└── Results
    ├── fastp
    ├── fastqc
    ├── featurecounts
    ├── logfiles
    ├── multiqc
    ├── qualimap
    ├── samtools
    └── star

17 directories
=== current working directory ===
/shared/ifbstor1/projects/gonseq/Testing/Pipeline


In [4]:
module load samtools subread stringtie

echo "===== bam sorting by names ====="
samtools --version | head -n 2
echo "===== gene level quantification ====="
featureCounts -v
echo "===== transcripts / isoforms level quantification ====="
stringtie --version

===== bam sorting by names =====
samtools 1.10
Using htslib 1.10.2
===== gene level quantification =====

featureCounts v2.0.1

===== transcripts / isoforms level quantification =====
2.1.2


---
## <b>I- Some checks as a precaution</b>

As we change session or even day, let's first check all files are there:

In [5]:
mappedfolder="${gohome}Results/star/"

echo "There are $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam | wc -l) bam files:"
ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam

There are 16 bam files:
/shared/projects/gonseq/Building/Results/star/SRR7430706_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430707_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430708_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430709_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430710_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430711_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430712_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430713_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430738_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430739_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430740_Aligned.sortedByCoord.out.bam
/shared/projects/go

## <b>II- Gene level quantification using `featureCounts`</b>

### **1- Tool discovery**

`featureCounts` is part of a package called <a href="http://subread.sourceforge.net/">SubRead</a>, to be used with bash, and RSubRead, to be used with R. This tool allows to attribute mapped reads to their matching feature (exon, gene, promoter, ...) on the genome and summarize counts per feature.  
  
In SubRead user guide, developpers recommend to use <i>specialized transcript-level quantification tools [...] for counting reads to transcripts</i> (see section 6.2.5, page 34 of pdf manual you can download with this <a href="http://bioinf.wehi.edu.au/subread-package/SubreadUsersGuide.pdf">link</a>). So we will only generate gene-level quantification with `featureCounts`.

In [ ]:
featureCounts -v

A simple usage to count paired-end sequencing data, then counts fragments instead or reads, is:  
<code>featureCounts -p -a annotation.gtf \ 
                  -o counts.txt \ 
                  alignment.bam</code>

The <code>feautureCounts</code> options correspond to: <br>
<blockquote>
    <code>-p</code>, to count fragments instead or reads (paired-end data) <br>
    <code>-a TEXT</code>, to locate annotation file, in GTF/GFF format by default (<code>-f</code> to be used to give other file format) that can be a <code>.gzip</code> one. <br>
    <code>-o TEXT</code>, to set filename for counts<br>
    then alignement files (it may be a list) come. Either in BAM or SAM format, they can be sorted by read names or by chromosomal coordinates.
</blockquote>

By default, multi-mapped reads (or fragments) are not considered unless we use `-M` option (see others parameters to set in manual, pdf pages 38 to 43).  
It will select <code>exon</code> lines in annotation file (<code>-T exon</code> by default) to attribute reads (or fragments). Then, it will count them according to <code>gene_id</code> meta-feature level (<code>-g gene_id</code> by default).

We will use some other options:  
<blockquote>
    <code>-s INTEGER</code>, to specify strandness. Possible values include: 0 (unstranded), 1 (stranded) and 2 (reversely stranded). Default value is 0. <br>
    <code>-T INTEGER</code>, to set the number of threads to use (default, 1) <br>
    <code>--verbose</code>, to get information for debugging, such as unmatched chromosome/contig names. <br>
    As temporary files are saved by default to directory specified in <code>-o</code> option, we won't use <code>--tmpDir STRING</code> option <br>
</blockquote>

For paired-end data, it is possible to ask for filtering fragments that have both ends aligned (`-B` option), on same chromose and strand (`-C`) and even separated by an insert distance (`-P`, included in `-d` and `-D` values).  

Besides, input files will be used as sorted by names. Even if `featureCounts` handles `.bam` files as fast as `samtools` does, we will nonetheless use `samtools` and `--donotsort` option. Indee, some files may be bigger than supported by `featureCounts`.

### **2- Step preparation**

We need to create a destination folder...

In [6]:
featcountfolder="${gohome}Results/featurecounts/"
mkdir -p ${featcountfolder}
tree -d -L 1 "${gohome}Results/"

/shared/projects/gonseq/Building/Results/
├── fastp
├── fastqc
├── featurecounts
├── logfiles
├── multiqc
├── qualimap
├── samtools
└── star

8 directories


... and remember used folder for saved screen output files.

In [7]:
logfolder="${gohome}Results/logfiles/"

### **3- Compute reads or fragments counts**

<div class="alert alert-block alert-danger">
    Following <b>command is prepared for usage on a computational cluster</b> and was developped on the <i>Institut Français de Bioinformatique</i> (IFB)'s core cluster. We use a <i>Large</i> session defined as <b>10 CPU with 50 GB available for RAM</b>. 
</div>

<div class="alert alert-block alert-warning">
    <b><code>-T</code> option in featureCounts command line doesn't seem to have impact during counting process</b>. It has been seen active when BAM are sorted (thus not used in current version of below cell).
</div>

In [10]:
logfile="${logfolder}featureCounts-gene-level-counts.log"
echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

time for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do  
    
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "===== Processing sampleID: ${id}..." | tee -a ${logfile}
    
    # outputfiles
    mytempfile="${featcountfolder}${id}_Aligned.sortedByNames.bam"
    myoutfile="${featcountfolder}${id}_paired-reverse-stranded"
    
    # bam sorting...
    echo "samtools starts at $(date)" >> ${logfile}
    samtools sort -n \
                --threads 8 -m 5G \
                --output-fmt BAM \
                -o ${mytempfile} \
                -T ${featcountfolder} \
                ${fn} \
                &>> ${logfile}
    echo "samtools ends at $(date)" >> ${logfile}

    # some user conversation to help being patient
    echo "changing tool" | tee -a ${logfile}

    # then featureCounts
    echo "featureCounts starts at $(date)" >> ${logfile}

    featureCounts -p -s 2 -T 8 \
                  -a "${gohome}Reference/extracted/genome_annotation.gtf" \
                  -o "${myoutfile}.counts" \
                  ${mytempfile} \
                  --donotsort \
                  --verbose \
                  &>> ${logfile}
    echo "featureCounts ends at $(date)" >> ${logfile}
    
    # removing extra bam file... saving disk space
    rm ${mytempfile}
    
    echo "... done" | tee -a ${logfile}
    
done

echo "operation ends at $(date)" >> ${logfile}

echo "=== Files created after featureCounts ===" >> ${logfile}
ls -lh "${featcountfolder}" >> ${logfile}
echo "featureCounts generated $(ls "${featcountfolder}"*.counts | wc -l) count files." \
    | tee -a ${logfile}
echo "featureCounts generated $(ls "${featcountfolder}"*.counts.summary | wc -l) summary files." \
    | tee -a ${logfile}

Screen output is redirected to /shared/projects/gonseq/Building/Results/logfiles/featureCounts-gene-level-counts.log
===== Processing sampleID: SRR7430706...
changing tool
... done
===== Processing sampleID: SRR7430707...
changing tool
... done
===== Processing sampleID: SRR7430708...
changing tool
... done
===== Processing sampleID: SRR7430709...
changing tool
... done
===== Processing sampleID: SRR7430710...
changing tool
... done
===== Processing sampleID: SRR7430711...
changing tool
... done
===== Processing sampleID: SRR7430712...
changing tool
... done
===== Processing sampleID: SRR7430713...
changing tool
... done
===== Processing sampleID: SRR7430738...
changing tool
... done
===== Processing sampleID: SRR7430739...
changing tool
... done
===== Processing sampleID: SRR7430740...
changing tool
... done
===== Processing sampleID: SRR7430741...
changing tool
... done
===== Processing sampleID: SRR7430742...
changing tool
... done
===== Processing sampleID: SRR7430743...
changing t

We can see the proportions of alignements that match a single gene (if default `featureCounts` parameters where kept) thanks to logfile and a research for specifiy patterns (`grep -e PATTERN`).

In [11]:
cat ${logfile} | grep -e "Successfully assigned alignments" -e "Process BAM"

|| Process BAM file SRR7430706_Aligned.sortedByNames.bam...                   ||
||    Successfully assigned alignments : 31270906 (47.2%)                     ||
|| Process BAM file SRR7430707_Aligned.sortedByNames.bam...                   ||
||    Successfully assigned alignments : 26695027 (47.0%)                     ||
|| Process BAM file SRR7430708_Aligned.sortedByNames.bam...                   ||
||    Successfully assigned alignments : 25233609 (38.1%)                     ||
|| Process BAM file SRR7430709_Aligned.sortedByNames.bam...                   ||
||    Successfully assigned alignments : 25735510 (43.6%)                     ||
|| Process BAM file SRR7430710_Aligned.sortedByNames.bam...                   ||
||    Successfully assigned alignments : 29774681 (41.0%)                     ||
|| Process BAM file SRR7430711_Aligned.sortedByNames.bam...                   ||
||    Successfully assigned alignments : 29244931 (49.5%)                     ||
|| Process BAM file SRR74307

In [ ]:
# to have explanations
echo "summary file"
cat "${myoutfile}.counts.summary"

In [ ]:
# to display assignment proportion
echo "screen output extract"
cat ${logfile} | grep "Successfully assigned alignments" | tail -n 1

## <b>III- Transcript level quantification using `stringtie`</b>

<div class="alert alert-block alert-warning">
    Part to be treated after M1 report and oral.
</div>